In [35]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [36]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [37]:
df_green.registerTempTable('green')

In [66]:
df_green_revenue = spark.sql("""
SELECT  
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    

    SUM(total_amount) amount,
    COUNT(1) as number_records

FROM
    green
WHERE
    lpep_pickup_datetime >='2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1,2
""")

In [67]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

23/03/08 00:50:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/08 00:50:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/08 00:50:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/08 00:50:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/08 00:50:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/08 00:50:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
23/03/08 00:50:48 WARN MemoryManager: Total allocation exceeds 95.

In [68]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [69]:
df_yellow.registerTempTable('yellow')

In [70]:
df_yellow_revenue = spark.sql("""
SELECT  
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    

    SUM(total_amount) amount,
    COUNT(1) as number_records

FROM
    yellow
WHERE
    tpep_pickup_datetime >='2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1,2
""")

In [71]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

23/03/08 00:50:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/08 00:50:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/08 00:50:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/08 00:50:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/08 00:50:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/08 00:50:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
23/03/08 00:50:52 WARN MemoryManager: Total allocation exceeds 95.

In [80]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount','green_amount') \
    .withColumnRenamed('number_records','green_number_records') 

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount','yellow_amount') \
    .withColumnRenamed('number_records','yellow_number_records') 

In [84]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [85]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp,on=['hour','zone'], how='outer')

In [87]:
df_join.write.parquet('data/report/revenue/total',mode='overwrite')

23/03/08 00:57:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/08 00:57:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/08 00:57:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/08 00:57:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/08 00:57:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/08 00:57:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
23/03/08 00:57:51 WARN MemoryManager: Total allocation exceeds 95.

In [83]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  22|              15.8|                   1|              null|                 null|
|2020-01-01 00:00:00|  62|             15.95|                   1|             61.43|                    1|
|2020-01-01 00:00:00|  65|            199.49|                  10| 409.3499999999999|                   19|
|2020-01-01 00:00:00|  92|             80.72|                   3|              75.3|                    1|
|2020-01-01 00:00:00| 134|  

In [91]:
df_join = spark.read.parquet('data/report/revenue/total')

In [92]:
df_zones = spark.read.parquet('zones/')

In [93]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [96]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [99]:
df_result.drop('LocationID','zone').write.parquet('tmp/revenue-zones')

23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.

23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 50.67% for 15 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 54.29% for 14 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/08 01:02:06 WARN MemoryManager: Total allocation exceeds 9